# Universidade de Fortaleza
## Mestrado em Informática Aplicada
### Ciência de Dados aplicada à Ciência da Cidade

# Isochrone Mapping for Urban Data Science

## Introduction

**Isochrone maps** (from Greek *iso* = equal, *chronos* = time) visualize areas reachable within the same travel time from a given location. While sharing the "equal time" concept with the mathematical isochronous curves (like the cycloid), urban isochrones are practical tools for analyzing accessibility in cities.

### Applications in Urban Planning:
- **Transit Accessibility**: Measure how well public transportation serves different neighborhoods
- **Emergency Services**: Identify areas reachable by ambulances within critical response times
- **Food Deserts**: Find neighborhoods lacking access to grocery stores
- **Employment Access**: Analyze job opportunities reachable within commute time budgets
- **Equity Analysis**: Compare accessibility across socioeconomic groups

### This Notebook

We'll explore isochrone analysis using:
1. **Simulated street networks** to understand the concept
2. **Walking distance calculations** using graph theory
3. **Multiple transportation modes** (walking, cycling, driving)
4. **Visualization techniques** for accessibility mapping
5. **Equity metrics** to compare accessibility across locations

## Setup and Dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.patches import Circle
from matplotlib.collections import LineCollection
import networkx as nx
from scipy.spatial import Voronoi, voronoi_plot_2d
from collections import defaultdict

# Set random seed for reproducibility
np.random.seed(42)

# Plot styling
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## Part 1: Creating a Simulated Street Network

We'll create a synthetic city grid to demonstrate isochrone concepts. This represents a simplified urban street network.

In [ ]:
def create_grid_network(width=10, height=10):
    """
    Create a grid-based street network.
    
    Parameters:
    - width: Number of blocks in x direction
    - height: Number of blocks in y direction
    
    Returns:
    - G: NetworkX graph representing the street network
    """
    G = nx.grid_2d_graph(width, height)
    
    # Add random edge weights representing travel time (in minutes)
    # Base time + random variation to simulate traffic/conditions
    for (u, v) in G.edges():
        # Manhattan distance between nodes
        distance = abs(u[0] - v[0]) + abs(u[1] - v[1])
        # Base travel time (e.g., 2 min per block) + random variation
        travel_time = distance * 2.0 + np.random.uniform(0, 1)
        G[u][v]['weight'] = travel_time
        G[u][v]['distance'] = distance * 100  # meters
    
    return G

# Create network
G = create_grid_network(15, 15)

# Visualize the network
plt.figure(figsize=(10, 10))
pos = {node: node for node in G.nodes()}
nx.draw(G, pos, node_size=50, node_color='lightblue', 
        edge_color='gray', width=0.5, with_labels=False)
plt.title('Simulated Street Network (Grid)')
plt.xlabel('X (blocks)')
plt.ylabel('Y (blocks)')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Network created: {G.number_of_nodes()} intersections, {G.number_of_edges()} streets")

## Part 2: Computing Isochrones

An isochrone shows all locations reachable within a given travel time. We'll use Dijkstra's shortest path algorithm to compute travel times from a starting point.

In [ ]:
def compute_isochrones(G, source, time_limits=[5, 10, 15, 20, 25]):
    """
    Compute isochrone zones from a source node.
    
    Parameters:
    - G: NetworkX graph
    - source: Starting node
    - time_limits: List of time thresholds in minutes
    
    Returns:
    - isochrones: Dict mapping time limits to sets of reachable nodes
    - travel_times: Dict mapping nodes to travel time from source
    """
    # Compute shortest paths using Dijkstra's algorithm
    travel_times = nx.single_source_dijkstra_path_length(G, source, weight='weight')
    
    # Group nodes by isochrone zones
    isochrones = {}
    for time_limit in time_limits:
        isochrones[time_limit] = {
            node for node, time in travel_times.items() 
            if time <= time_limit
        }
    
    return isochrones, travel_times

# Choose a starting point (center of the grid)
source_node = (7, 7)

# Compute isochrones
time_limits = [5, 10, 15, 20, 25]
isochrones, travel_times = compute_isochrones(G, source_node, time_limits)

# Display statistics
print(f"Starting point: {source_node}\n")
print("Isochrone Coverage:")
for time_limit in time_limits:
    count = len(isochrones[time_limit])
    percentage = (count / G.number_of_nodes()) * 100
    print(f"  {time_limit} min: {count} locations ({percentage:.1f}% of network)")

## Part 3: Visualizing Isochrones

Let's create a color-coded map showing travel time zones.

In [ ]:
def visualize_isochrones(G, source, isochrones, time_limits):
    """
    Visualize isochrone zones with color coding.
    """
    fig, ax = plt.subplots(figsize=(12, 12))
    
    # Color map for different time zones
    colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.9, len(time_limits)))
    
    # Draw nodes colored by isochrone zone
    pos = {node: node for node in G.nodes()}
    
    # First, draw all edges in light gray
    nx.draw_networkx_edges(G, pos, edge_color='lightgray', width=0.5, ax=ax)
    
    # Draw nodes for each isochrone zone
    for i, time_limit in enumerate(time_limits):
        nodes_in_zone = isochrones[time_limit]
        if i > 0:
            # Remove nodes from previous zones
            nodes_in_zone = nodes_in_zone - isochrones[time_limits[i-1]]
        
        nx.draw_networkx_nodes(G, pos, nodelist=list(nodes_in_zone),
                               node_color=[colors[i]], node_size=100, ax=ax,
                               label=f'{time_limits[i-1] if i > 0 else 0}-{time_limit} min')
    
    # Highlight source node
    nx.draw_networkx_nodes(G, pos, nodelist=[source], node_color='blue',
                           node_size=300, node_shape='*', ax=ax, label='Starting Point')
    
    ax.set_title(f'Isochrone Map from Starting Point {source}', fontsize=14, fontweight='bold')
    ax.set_xlabel('X (blocks)')
    ax.set_ylabel('Y (blocks)')
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()

visualize_isochrones(G, source_node, isochrones, time_limits)

## Part 4: Multiple Transportation Modes

Different modes of transportation have different speeds. Let's compare walking, cycling, and driving.

In [ ]:
def create_multimodal_network(width=15, height=15):
    """
    Create networks for different transportation modes.
    
    Average speeds:
    - Walking: 5 km/h (83 m/min)
    - Cycling: 15 km/h (250 m/min)
    - Driving: 30 km/h in city (500 m/min)
    """
    modes = {
        'walking': {'speed': 83, 'color': 'green'},
        'cycling': {'speed': 250, 'color': 'orange'},
        'driving': {'speed': 500, 'color': 'red'}
    }
    
    networks = {}
    
    for mode, config in modes.items():
        G = nx.grid_2d_graph(width, height)
        
        for (u, v) in G.edges():
            # Distance in meters (100m per block)
            distance = (abs(u[0] - v[0]) + abs(u[1] - v[1])) * 100
            
            # Travel time = distance / speed
            travel_time = distance / config['speed']
            
            # Add some randomness (traffic, conditions)
            travel_time *= np.random.uniform(0.9, 1.1)
            
            G[u][v]['weight'] = travel_time
            G[u][v]['distance'] = distance
        
        networks[mode] = G
    
    return networks, modes

# Create multimodal networks
networks, modes = create_multimodal_network()

# Compute isochrones for each mode (15-minute travel time)
source = (7, 7)
comparison_time = 15

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, (mode, G) in enumerate(networks.items()):
    isochrones, travel_times = compute_isochrones(G, source, [comparison_time])
    reachable_nodes = isochrones[comparison_time]
    
    # Visualize
    ax = axes[idx]
    pos = {node: node for node in G.nodes()}
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, edge_color='lightgray', width=0.3, ax=ax)
    
    # Draw unreachable nodes
    unreachable = set(G.nodes()) - reachable_nodes
    nx.draw_networkx_nodes(G, pos, nodelist=list(unreachable),
                           node_color='lightgray', node_size=30, ax=ax)
    
    # Draw reachable nodes
    nx.draw_networkx_nodes(G, pos, nodelist=list(reachable_nodes),
                           node_color=modes[mode]['color'], node_size=50, ax=ax, alpha=0.7)
    
    # Draw source
    nx.draw_networkx_nodes(G, pos, nodelist=[source],
                           node_color='blue', node_size=200, node_shape='*', ax=ax)
    
    coverage = (len(reachable_nodes) / G.number_of_nodes()) * 100
    ax.set_title(f'{mode.capitalize()}\n{len(reachable_nodes)} locations ({coverage:.1f}%)',
                 fontweight='bold')
    ax.set_xlabel('X (blocks)')
    ax.set_ylabel('Y (blocks)')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')

fig.suptitle(f'{comparison_time}-Minute Accessibility by Transportation Mode', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## Part 5: Accessibility Equity Analysis

Let's analyze how accessibility varies across different starting locations, simulating an equity study.

In [ ]:
def compute_accessibility_scores(G, time_threshold=15):
    """
    Compute accessibility score for each node in the network.
    Score = number of locations reachable within time threshold.
    """
    accessibility = {}
    
    for node in G.nodes():
        travel_times = nx.single_source_dijkstra_path_length(G, node, weight='weight')
        reachable = sum(1 for time in travel_times.values() if time <= time_threshold)
        accessibility[node] = reachable
    
    return accessibility

# Compute accessibility for walking mode
print("Computing accessibility scores for all locations...")
G_walk = networks['walking']
accessibility = compute_accessibility_scores(G_walk, time_threshold=15)

# Convert to arrays for visualization
nodes = list(G_walk.nodes())
x = [n[0] for n in nodes]
y = [n[1] for n in nodes]
scores = [accessibility[n] for n in nodes]

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Heatmap of accessibility
scatter = ax1.scatter(x, y, c=scores, s=200, cmap='RdYlGn', 
                      vmin=min(scores), vmax=max(scores), edgecolors='black', linewidth=0.5)
ax1.set_title('Accessibility Heatmap\n(Walking, 15-minute threshold)', fontweight='bold')
ax1.set_xlabel('X (blocks)')
ax1.set_ylabel('Y (blocks)')
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')
cbar = plt.colorbar(scatter, ax=ax1)
cbar.set_label('Number of Reachable Locations', rotation=270, labelpad=20)

# Distribution of accessibility scores
ax2.hist(scores, bins=20, color='steelblue', edgecolor='black', alpha=0.7)
ax2.axvline(np.mean(scores), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(scores):.1f}')
ax2.axvline(np.median(scores), color='green', linestyle='--', linewidth=2, label=f'Median: {np.median(scores):.1f}')
ax2.set_title('Distribution of Accessibility Scores', fontweight='bold')
ax2.set_xlabel('Number of Reachable Locations')
ax2.set_ylabel('Frequency')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Statistics
print("\nAccessibility Statistics (15-minute walking):")
print(f"  Mean: {np.mean(scores):.2f} locations")
print(f"  Median: {np.median(scores):.2f} locations")
print(f"  Std Dev: {np.std(scores):.2f}")
print(f"  Min: {min(scores)} locations (worst accessibility)")
print(f"  Max: {max(scores)} locations (best accessibility)")
print(f"  Range: {max(scores) - min(scores)} locations")
print(f"\n  Equity Gap: {((max(scores) - min(scores)) / max(scores) * 100):.1f}%")

## Part 6: Service Coverage Analysis

Let's simulate placing essential services (e.g., hospitals, schools) and analyze how well they serve the population.

In [ ]:
def analyze_service_coverage(G, service_locations, max_time=10):
    """
    Analyze how well service locations cover the network.
    
    Parameters:
    - G: Network graph
    - service_locations: List of service node locations
    - max_time: Maximum acceptable travel time to service
    
    Returns:
    - coverage_map: Dict mapping each node to nearest service and travel time
    """
    coverage_map = {}
    
    for node in G.nodes():
        min_time = float('inf')
        nearest_service = None
        
        for service in service_locations:
            try:
                time = nx.shortest_path_length(G, node, service, weight='weight')
                if time < min_time:
                    min_time = time
                    nearest_service = service
            except nx.NetworkXNoPath:
                continue
        
        coverage_map[node] = {
            'nearest_service': nearest_service,
            'travel_time': min_time,
            'is_covered': min_time <= max_time
        }
    
    return coverage_map

# Place services strategically (e.g., hospitals)
# Strategy 1: Center placement
services_center = [(7, 7)]

# Strategy 2: Distributed placement
services_distributed = [(3, 3), (3, 11), (11, 3), (11, 11), (7, 7)]

# Analyze both strategies
G_walk = networks['walking']
max_acceptable_time = 10  # 10-minute maximum walk

coverage_center = analyze_service_coverage(G_walk, services_center, max_acceptable_time)
coverage_distributed = analyze_service_coverage(G_walk, services_distributed, max_acceptable_time)

# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

pos = {node: node for node in G_walk.nodes()}

for ax, coverage, services, title in [
    (ax1, coverage_center, services_center, 'Strategy 1: Center Placement'),
    (ax2, coverage_distributed, services_distributed, 'Strategy 2: Distributed Placement')
]:
    # Draw network
    nx.draw_networkx_edges(G_walk, pos, edge_color='lightgray', width=0.3, ax=ax)
    
    # Color nodes by coverage status
    covered = [n for n in G_walk.nodes() if coverage[n]['is_covered']]
    uncovered = [n for n in G_walk.nodes() if not coverage[n]['is_covered']]
    
    nx.draw_networkx_nodes(G_walk, pos, nodelist=covered,
                           node_color='lightgreen', node_size=50, ax=ax, label='Covered')
    nx.draw_networkx_nodes(G_walk, pos, nodelist=uncovered,
                           node_color='lightcoral', node_size=50, ax=ax, label='Not Covered')
    
    # Draw service locations
    nx.draw_networkx_nodes(G_walk, pos, nodelist=services,
                           node_color='blue', node_size=300, node_shape='s',
                           ax=ax, label='Service Location', edgecolors='black', linewidths=2)
    
    # Calculate statistics
    coverage_pct = (len(covered) / G_walk.number_of_nodes()) * 100
    avg_time = np.mean([coverage[n]['travel_time'] for n in G_walk.nodes() 
                        if coverage[n]['travel_time'] != float('inf')])
    
    ax.set_title(f'{title}\n{len(covered)}/{G_walk.number_of_nodes()} locations covered ({coverage_pct:.1f}%)\nAvg travel time: {avg_time:.2f} min',
                 fontweight='bold')
    ax.set_xlabel('X (blocks)')
    ax.set_ylabel('Y (blocks)')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')

fig.suptitle(f'Service Coverage Comparison\n(Maximum acceptable walking time: {max_acceptable_time} minutes)',
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

# Print detailed statistics
print("\nService Coverage Analysis:")
print("="*60)

for strategy, coverage, services in [
    ('Center Placement', coverage_center, services_center),
    ('Distributed Placement', coverage_distributed, services_distributed)
]:
    covered = sum(1 for v in coverage.values() if v['is_covered'])
    total = len(coverage)
    coverage_pct = (covered / total) * 100
    
    times = [v['travel_time'] for v in coverage.values() if v['travel_time'] != float('inf')]
    avg_time = np.mean(times)
    max_time = max(times)
    
    print(f"\n{strategy}:")
    print(f"  Number of services: {len(services)}")
    print(f"  Locations covered: {covered}/{total} ({coverage_pct:.1f}%)")
    print(f"  Average travel time: {avg_time:.2f} minutes")
    print(f"  Maximum travel time: {max_time:.2f} minutes")

## Part 7: Temporal Dynamics - Rush Hour vs Off-Peak

Travel times change throughout the day due to traffic congestion. Let's simulate this effect.

In [ ]:
def create_temporal_network(base_network, time_of_day='off_peak'):
    """
    Modify network to reflect different traffic conditions.
    
    Parameters:
    - base_network: Base network graph
    - time_of_day: 'rush_hour' or 'off_peak'
    """
    G = base_network.copy()
    
    if time_of_day == 'rush_hour':
        # Increase travel times by 50-150% during rush hour
        for (u, v) in G.edges():
            G[u][v]['weight'] *= np.random.uniform(1.5, 2.5)
    elif time_of_day == 'off_peak':
        # Slight reduction during off-peak
        for (u, v) in G.edges():
            G[u][v]['weight'] *= np.random.uniform(0.8, 1.0)
    
    return G

# Create networks for different times
G_base = networks['driving']
G_rush = create_temporal_network(G_base, 'rush_hour')
G_offpeak = create_temporal_network(G_base, 'off_peak')

# Compare 15-minute isochrones
source = (7, 7)
time_limit = 15

iso_rush, _ = compute_isochrones(G_rush, source, [time_limit])
iso_offpeak, _ = compute_isochrones(G_offpeak, source, [time_limit])

# Visualize
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 7))

pos = {node: node for node in G_base.nodes()}

for ax, G, isochrones, title in [
    (ax1, G_rush, iso_rush, 'Rush Hour'),
    (ax2, G_offpeak, iso_offpeak, 'Off-Peak'),
    (ax3, G_base, None, 'Difference')
]:
    nx.draw_networkx_edges(G, pos, edge_color='lightgray', width=0.3, ax=ax)
    
    if title != 'Difference':
        reachable = isochrones[time_limit]
        unreachable = set(G.nodes()) - reachable
        
        nx.draw_networkx_nodes(G, pos, nodelist=list(unreachable),
                               node_color='lightgray', node_size=30, ax=ax)
        nx.draw_networkx_nodes(G, pos, nodelist=list(reachable),
                               node_color='coral', node_size=50, ax=ax, alpha=0.7)
        
        coverage = (len(reachable) / G.number_of_nodes()) * 100
        ax.set_title(f'{title}\n{len(reachable)} locations ({coverage:.1f}%)', fontweight='bold')
    else:
        # Show difference: areas reachable off-peak but not during rush hour
        rush_only = iso_rush[time_limit]
        offpeak_only = iso_offpeak[time_limit]
        both = rush_only & offpeak_only
        offpeak_advantage = offpeak_only - rush_only
        
        nx.draw_networkx_nodes(G, pos, nodelist=list(both),
                               node_color='lightgreen', node_size=30, ax=ax, label='Both')
        nx.draw_networkx_nodes(G, pos, nodelist=list(offpeak_advantage),
                               node_color='darkgreen', node_size=80, ax=ax, label='Off-peak advantage')
        
        ax.set_title(f'Impact of Traffic\n{len(offpeak_advantage)} additional locations reachable off-peak',
                     fontweight='bold')
        ax.legend()
    
    nx.draw_networkx_nodes(G, pos, nodelist=[source],
                           node_color='blue', node_size=200, node_shape='*', ax=ax)
    
    ax.set_xlabel('X (blocks)')
    ax.set_ylabel('Y (blocks)')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')

fig.suptitle(f'{time_limit}-Minute Driving Accessibility: Temporal Comparison',
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

# Statistics
print("\nTemporal Analysis:")
print(f"  Rush hour coverage: {len(iso_rush[time_limit])} locations")
print(f"  Off-peak coverage: {len(iso_offpeak[time_limit])} locations")
print(f"  Difference: {len(iso_offpeak[time_limit]) - len(iso_rush[time_limit])} locations")
print(f"  Impact: {((len(iso_offpeak[time_limit]) - len(iso_rush[time_limit])) / len(iso_rush[time_limit]) * 100):.1f}% more coverage off-peak")

## Conclusions and Urban Planning Insights

### Key Findings from This Analysis:

1. **Transportation Mode Matters**: Faster transportation dramatically expands the area reachable within a given time budget.

2. **Location Inequality**: Central locations typically have better accessibility than peripheral ones, creating equity concerns.

3. **Service Distribution Strategy**: Distributed service locations provide better coverage than centralized ones, especially for walking access.

4. **Temporal Variations**: Traffic congestion during rush hours can reduce accessibility by 20-40%, affecting commute patterns.

### Applications in Urban Planning:

- **Transit Planning**: Identify underserved neighborhoods that need better public transportation
- **Facility Location**: Optimize placement of schools, hospitals, grocery stores for equitable access
- **Housing Policy**: Understand how accessibility affects property values and housing affordability
- **Infrastructure Investment**: Prioritize road/transit improvements that maximize accessibility gains
- **Environmental Justice**: Ensure disadvantaged communities have equal access to opportunities

### Connection to Isochronous Curves:

While the mathematical isochronous curve (cycloid) ensures objects reach a destination in equal time from any starting height, urban isochrone maps show the *opposite* reality: from a single starting point, different destinations require vastly different travel times. This makes isochrone analysis a powerful tool for identifying and addressing urban inequities.

### Next Steps:

To extend this analysis with real-world data:
- Use OpenStreetMap data via `osmnx` library for actual city street networks
- Integrate real-time traffic data from APIs (Google Maps, HERE, TomTom)
- Add demographic data to analyze accessibility by income, age, or other factors
- Include transit schedules for multimodal journey planning
- Perform 3D analysis considering elevation and walking difficulty

## Exercise Questions

1. How would you modify the network to include public transit (buses with fixed routes and schedules)?

2. If you had to place exactly 3 emergency service stations to maximize coverage within 5 minutes, where would you place them?

3. How could you quantify the "accessibility gap" between the most and least accessible locations? Design a metric.

4. What factors beyond travel time should be considered when measuring true accessibility (cost, safety, comfort, etc.)?

5. How might climate change (extreme heat, flooding) affect the isochrone maps, especially for walking and cycling?